База данных (БД) - хранилище данных.

Система управления базой данных (СУБД) - программа, которая позволяет работать с данными с помощью запросов.

### Типы баз данных

* Самым распространенным типом является реляционная база данных (SQL), от слова relation — отношение. Примеры: SQLite, MySQL, MSSQL, [PostgreSQL](https://www.postgresql.org/).
* Следом по популярности идут объектные базы данных. Данные в них хранятся в виде объектов, как в Объектно-Ориентированном Программировании.
Примеры: [ObjectDatabase++](http://tdbe.ekkysoftware.com/ODBPP)
* Существуют документоориентированные базы данных (NoSQL). Самый популярный пример — [MongoDB](https://www.mongodb.com/). Например, в таких базах можно хранить документы форматов XML и json.
* Другой пример — графовые базы данных. Они позволяют хранить данные в виде графа, имеющего объекты и связи между ними. Такие базы удобно использовать, например, для хранения геоданных или данных о пользователях в социальной сети. Пример — [Neo4J](https://neo4j.com/).

Еще один интересный, с точки зрения анализа данных, пример — столбцовые (колоночные) БД. Например, [ClickHouse от Яндекса](https://habr.com/ru/company/yandex/blog/303282/). Попробовать [online](https://clickhouse.tech). Эту СУБД используют в Яндексе для хранения данных из Яндекс.Метрики, так как она позволяет быстро сохранять слабо структурированные данные в базу и обрабатывать огромные объёмы данных в реальном времени. Важно помнить, что если при обработке данных нам приходится больше работать со строками, а не со столбцами, то лучше использовать реляционные БД.

### О реляционных базах данных

Реляционная база данных — это набор таблиц, связанных друг с другом. При этом каждая таблица хранит информацию об объектах определённого типа. Например, в одной таблице будут данные о людях, а в другой — об их домашних животных. 

Строка таблицы содержит данные об одном объекте (например, о человеке из первой базы). Столбцы таблицы описывают различные характеристики (атрибуты) этих объектов. Например, имя, пол, возраст и так далее. Каждый атрибут имеет строго определённый тип данных. На столбец можно накладывать ограничение NOT NULL, говорящее, что в нем не должно быть пропусков. Тогда система не даст добавить запись с пропуском в данном атрибуте.

Записи (примеры, строки) имеют одинаковую структуру: они состоят из полей, хранящих атрибуты (признаки) объекта. В строках могут находиться пропуски, если только не наложено ограничение NOT NULL.

Хорошая подробная статья о том, [как работает реляционная БД](https://habr.com/ru/company/yandex/blog/303282/).

### Связи в таблицах

Таблицы в  могут быть связаны друг с другом. Две таблицы связаны, если в них имеются одинаковые столбцы.

В реляционной базе данных каждая таблица должна иметь первичный ключ (PRIMARY KEY) — поле или комбинацию полей, которые единственным образом идентифицируют каждую строку таблицы.

Если ключ состоит из нескольких полей, то он называется составным. Ключ должен быть уникальным и однозначно определять запись. По значению ключа можно отыскать единственную запись. Ключи служат также для упорядочивания информации в БД. В качестве ключа можно использовать какой-то искусственно созданный идентификатор человека или уже существующие уникальные коды типа ИНН. Нехорошо выбирать идентификатором имя, так как оно не уникально. Несмотря на то, что ключом можно объявить сразу несколько атрибутов, выделять атрибуты (фамилия, имя, отчество) как составной первичный ключ тоже нехорошо, по той же причине.

Кроме первичного ключа существует внешний ключ (FOREIGN KEY), который создаёт жесткую связь между парой таблиц. Как это работает? Создается Таблица 1, в которой определяется первичный ключ (например, id). Затем создается Таблица 2, в которой определяется внешний ключ (например, f_id), при этом явно указывается, что он ссылается на первичный ключ из таблицы 1 (который мы назвали id).

Такая связь позволяет при изменении значения первичного ключа в первой таблице изменить все соответствующие значения связанного поля во второй таблице. При удалении записи с определенным первичным ключом в первой таблице могут быть удалены все записи с определенным (тем же) значением связанного внешним ключом поля во второй таблице.

Например, у нас есть таблица с данными о сотрудниках. Кроме этого, есть таблица с медицинскими данными для каждого сотрудника, для которого есть такая информация. Во второй таблице будет указан внешний ключ, ссылающийся на идентификатор сотрудника в первой таблице. При удалении сотрудника можно будет автоматически (каскадно) удалить соответствующие ему записи из других связанных таблиц, в том числе, из таблицы с медицинскими данными.

### Что такое нормализация?

Часто говорят, что таблицы реляционной БД должны отвечать требованиям нормализации отношений.

Нормализация — это способ наложить ограничение на формирование таблиц, которое позволяет устранить дублирование, обеспечивает непротиворечивость хранимых в базе данных, уменьшает трудозатраты на ведение базы данных.

Чтобы нормализовать таблицу, ее приводят к нормальным формам. Подробнее [про нормальные формы на Хабре](https://habr.com/ru/post/254773/).

In [28]:
import sqlite3
conn = sqlite3.connect('restaurant.db') # Создаем соединение с нашей базой данных

# Часть №1

#### Хотим таблицу Склад:

| Название ингредиента | Количество | Дата* |
| -------------------- |:----------:|:-----:|
| Брокколи             | 25         | 02.12 |
| Куриное филе         | 5          | 03.12 |
| Брокколи             | 30         | 04.12 |

*когда испортится или когда привезли

#### Как ее правильно реализовать в реляционных базах данных?
###### Таблица Склад:  
   
| id | id ингредиента | Количество | Дата* |
| -- | -------------- |:----------:|:-----:|
| 1  | 5              | 25         | 1575314617 |
| 2  | 7              | 5          | 1575401017 |
| 3  | 5              | 30         | 1575487417 |

###### Таблица Ингредиенты:

| id ингредиента | Название |
| -- | -------------- |
| 5  | Брокколи       |
| 7  | Куриное филе   |

*unix-time

In [29]:
# Создаем курсор - это специальный объект который делает запросы и получает их результаты
c = conn.cursor()

# Создаем таблицу ингредиентов
c.execute('''CREATE TABLE ingredients
             (id INTEGER PRIMARY KEY, name TEXT)''')

# Создаем таблицу для склада
c.execute('''CREATE TABLE warehouse
             (id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ingredient_id INTEGER, count INTEGER, date REAL)''')

# Save (commit) the changes
conn.commit()

Заполним таблицу ингредиентов:

In [5]:
ingredients = ['Молоко', 'Яйцо', 'Мука', 'Разрыхлитель', 'Масло растительное', 'Сахар', 'Соль']
for i in map(lambda it: (it, ), ingredients):
    print(i)
    c.execute("INSERT INTO ingredients (name) VALUES (?)", i)

('Молоко',)
('Яйцо',)
('Мука',)
('Разрыхлитель',)
('Масло растительное',)
('Сахар',)
('Соль',)


In [33]:
c.execute("SELECT * FROM ingredients")
results = c.fetchall()
results

[(1, 'Молоко')]

In [7]:
c.execute("SELECT name FROM ingredients")
results = c.fetchall()
results

[('Молоко',),
 ('Яйцо',),
 ('Мука',),
 ('Разрыхлитель',),
 ('Масло растительное',),
 ('Сахар',),
 ('Соль',)]

In [8]:
c.execute("SELECT * FROM ingredients WHERE id=5")
print(c.fetchall())

c.execute("SELECT * FROM ingredients WHERE name='Соль'")
print(c.fetchall())

[(5, 'Масло растительное')]
[(7, 'Соль')]


Посчитаем количество строк этой таблицы:

In [9]:
c.execute("SELECT COUNT(*) FROM ingredients")
print(c.fetchall())

c.execute("SELECT SUM(id) FROM ingredients")
print(c.fetchall())

[(7,)]
[(28,)]


Создадим случайные данные для Склада и запишем их в БД:

In [10]:
c.execute("SELECT id FROM ingredients")
results = c.fetchall()
print(results)
ids = list(map(lambda it: it[0], results))
print(ids)

[(1,), (2,), (3,), (4,), (5,), (6,), (7,)]
[1, 2, 3, 4, 5, 6, 7]


In [11]:
import random
import datetime

for i in range(1, 28):
    count = random.randint(10, 100)
    date = datetime.datetime(2019, 11, i, hour=random.randint(12, 18), minute=random.randint(0, 59), second=0, microsecond=0, tzinfo=None)
    ingredient = random.choice(ids)
    c.execute("INSERT INTO warehouse (ingredient_id, count, date) VALUES (?, ?, ?)", (ingredient, count, date.timestamp()))

In [12]:
c.execute("SELECT * FROM warehouse")
results = c.fetchall()
results

[(1, 4, 62, 1572611520.0),
 (2, 3, 14, 1572696720.0),
 (3, 4, 27, 1572786240.0),
 (4, 2, 21, 1572859740.0),
 (5, 6, 96, 1572968760.0),
 (6, 4, 84, 1573050540.0),
 (7, 6, 76, 1573125120.0),
 (8, 3, 16, 1573215180.0),
 (9, 4, 71, 1573291320.0),
 (10, 5, 63, 1573394760.0),
 (11, 2, 53, 1573466460.0),
 (12, 5, 87, 1573554480.0),
 (13, 4, 95, 1573649640.0),
 (14, 6, 97, 1573739460.0),
 (15, 2, 62, 1573814280.0),
 (16, 4, 65, 1573902000.0),
 (17, 1, 67, 1574005920.0),
 (18, 3, 54, 1574092500.0),
 (19, 2, 32, 1574172060.0),
 (20, 2, 31, 1574262420.0),
 (21, 2, 27, 1574335380.0),
 (22, 7, 40, 1574433180.0),
 (23, 3, 42, 1574516220.0),
 (24, 7, 14, 1574605260.0),
 (25, 6, 63, 1574674140.0),
 (26, 1, 53, 1574780820.0),
 (27, 6, 19, 1574850000.0)]

Приведем все в нормальный вид:

In [13]:
def prettyWarehouse(cursor):
    cursor.execute("""SELECT name, count, date FROM warehouse
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id""")
    return cursor.fetchall()

prettyWarehouse(c)

[('Разрыхлитель', 62, 1572611520.0),
 ('Мука', 14, 1572696720.0),
 ('Разрыхлитель', 27, 1572786240.0),
 ('Яйцо', 21, 1572859740.0),
 ('Сахар', 96, 1572968760.0),
 ('Разрыхлитель', 84, 1573050540.0),
 ('Сахар', 76, 1573125120.0),
 ('Мука', 16, 1573215180.0),
 ('Разрыхлитель', 71, 1573291320.0),
 ('Масло растительное', 63, 1573394760.0),
 ('Яйцо', 53, 1573466460.0),
 ('Масло растительное', 87, 1573554480.0),
 ('Разрыхлитель', 95, 1573649640.0),
 ('Сахар', 97, 1573739460.0),
 ('Яйцо', 62, 1573814280.0),
 ('Разрыхлитель', 65, 1573902000.0),
 ('Молоко', 67, 1574005920.0),
 ('Мука', 54, 1574092500.0),
 ('Яйцо', 32, 1574172060.0),
 ('Яйцо', 31, 1574262420.0),
 ('Яйцо', 27, 1574335380.0),
 ('Соль', 40, 1574433180.0),
 ('Мука', 42, 1574516220.0),
 ('Соль', 14, 1574605260.0),
 ('Сахар', 63, 1574674140.0),
 ('Молоко', 53, 1574780820.0),
 ('Сахар', 19, 1574850000.0)]

Посчитаем количество ингредиентов на складе:

In [14]:
def ingredientsOnWarehouse(cursor):
    cursor.execute("""SELECT name, SUM(count) FROM warehouse 
          JOIN ingredients WHERE warehouse.ingredient_id = ingredients.id
          GROUP BY name""")
    return cursor.fetchall()

ingredientsOnWarehouse(c)

[('Масло растительное', 150),
 ('Молоко', 120),
 ('Мука', 126),
 ('Разрыхлитель', 404),
 ('Сахар', 351),
 ('Соль', 54),
 ('Яйцо', 226)]

Создадим функцию для поставки ингредиентов:

In [15]:
c.execute("SELECT id FROM ingredients WHERE name=?", ('Мука',))
results = c.fetchall()
print(results)
ingredient_id = results[0][0]
ingredient_id

[(3,)]


3

In [16]:
c.execute("SELECT id FROM ingredients WHERE name=?", ('Авокадо',))
results = c.fetchall()
print(results)
print(len(results))

[]
0


In [17]:
def getIngredientId(cursor, name):
    cursor.execute("SELECT id FROM ingredients WHERE name=?", (name, ))
    results = cursor.fetchall()
    if(len(results) > 0):
        return results[0][0]
    else:
        cursor.execute("INSERT INTO ingredients (name) VALUES (?)", (name, ))
        return getIngredientId(cursor, name)

In [18]:
import datetime
def supply(cursor, ingredient_name, count, date):
    ingredient_id = getIngredientId(cursor, ingredient_name)
    cursor.execute("INSERT INTO warehouse (ingredient_id, count, date) VALUES (?, ?, ?)", 
                       (ingredient_id, count, date.timestamp()))

In [19]:
c.execute("DELETE FROM warehouse");
c.execute("SELECT * FROM warehouse")
results = c.fetchall()
results

[]

In [20]:
c.execute("SELECT * FROM ingredients")
results = c.fetchall()
results

[(1, 'Молоко'),
 (2, 'Яйцо'),
 (3, 'Мука'),
 (4, 'Разрыхлитель'),
 (5, 'Масло растительное'),
 (6, 'Сахар'),
 (7, 'Соль')]

In [21]:
supply(c, 'Мука', 10, datetime.datetime.now())
supply(c, 'Авокадо', 10, datetime.datetime.now())

In [22]:
prettyWarehouse(c)

[('Мука', 10, 1588688351.069826), ('Авокадо', 10, 1588688351.071124)]

In [23]:
c.execute("SELECT * FROM ingredients")
results = c.fetchall()
results

[(1, 'Молоко'),
 (2, 'Яйцо'),
 (3, 'Мука'),
 (4, 'Разрыхлитель'),
 (5, 'Масло растительное'),
 (6, 'Сахар'),
 (7, 'Соль'),
 (8, 'Авокадо')]

In [34]:
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

### SQLAlchemy

In [24]:
# https://docs.sqlalchemy.org/en/13/dialects/sqlite.html